In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from statsmodels import regression

In [21]:
datastocks = pd.read_csv('data_snp_stocks.csv')
snp500 = pd.read_csv('sep500.csv', delimiter=';', usecols = ['Date', 'Price'])
snp500['Date'] = pd.to_datetime(snp500['Date'], format='mixed')
datastocks['date'] = pd.to_datetime(datastocks['date'], format = 'mixed')
datastocks.set_index('date', inplace=True)
tickers = datastocks['Name'].unique()

In [22]:
snp500['Price'].replace(',', '', regex=True, inplace=True)
snp500['Price'] = pd.to_numeric(snp500['Price'], errors='coerce')
snp500['Price'].dtypes

/var/folders/yc/zlgndsw96yb_kpncypv26x2h0000gn/T/ipykernel_1840/2474544338.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  snp500['Price'].replace(',', '', regex=True, inplace=True)


dtype('float64')

In [4]:
datastocks.dtypes

open      float64
high      float64
low       float64
close     float64
volume      int64
Name       object
dtype: object

In [5]:
datastocks.head()

,open,high,low,close,volume,Name
date,,,,,,
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [6]:
datastocks.isna().sum()

open      11
high       8
low        8
close      0
volume     0
Name       0
dtype: int64

In [7]:
#data.sort_values(by=['Name', 'date'], inplace=True)
datastocks['LogReturn'] = datastocks.groupby('Name')['close'].transform(lambda x: np.log(x / x.shift(1)))

In [8]:
datastocks['LogReturn'].isna().sum()

505

In [9]:
datastocks = datastocks[datastocks['LogReturn'].notna()]

In [10]:
start_date = '2017-01-01'
# Convert start_date to a datetime object
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')

# Add 180 days to start_date_obj
end_date_obj = start_date_obj + timedelta(days=261)

# Convert end_date_obj back to a string
end_date = end_date_obj.strftime('%Y-%m-%d')
#end_date = 

In [11]:
data180 = pd.DataFrame()
for ticker in tickers:
    data180 = pd.concat([data180, datastocks[(datastocks['Name'] == ticker) & (datastocks.index >= start_date) & (datastocks.index <= end_date)]])
data180

,open,high,low,close,volume,Name,LogReturn
date,,,,,,,
2017-01-03,47.28,47.340,46.135,46.30,6737752,AAL,-0.008388
2017-01-04,46.63,47.435,46.350,46.70,5859604,AAL,0.008602
2017-01-05,46.52,46.930,45.610,45.89,6825316,AAL,-0.017497
2017-01-06,45.85,46.720,45.470,46.21,7260197,AAL,0.006949
2017-01-09,46.01,47.340,45.780,47.08,4739142,AAL,0.018652
...,...,...,...,...,...,...,...
2017-09-13,65.32,65.830,65.220,65.28,1517180,ZTS,-0.001837
2017-09-14,65.10,65.270,64.770,65.25,2584337,ZTS,-0.000460
2017-09-15,65.41,65.640,64.870,65.20,2115398,ZTS,-0.000767


To perform the rolling regression we would need to solve the following equation:

$r_i -r_f = \alpha_i + \beta_i (R_M - r_f) + e_i$

Where:

$r_i$ is the return of the stock $i$ \
$r_f$ is the return of the risk free asset \
$\alpha_i$ is the intercept of the regression, or the excess return \
$\beta_i$ is the regression coefficient \
$R_M$ is the return of the market (log-return) \
$e_i$ is the specific risk of stock $i$


In [12]:
data180.groupby('Name').count()

,open,high,low,close,volume,LogReturn
Name,,,,,,
A,180,180,180,180,180,180
AAL,180,180,180,180,180,180
AAP,180,180,180,180,180,180
AAPL,180,180,180,180,180,180
ABBV,180,180,180,180,180,180
...,...,...,...,...,...,...
XYL,180,180,180,180,180,180
YUM,180,180,180,180,180,180
ZBH,180,180,180,180,180,180


In [23]:
snp500 = snp500[(snp500['Date'] >= start_date) & (snp500['Date'] <= end_date)].iloc[::-1]


In [24]:
snp500

,Date,Price
1878,2017-01-03,2.258
1877,2017-01-04,2.271
1876,2017-01-05,2.269
1875,2017-01-06,2.277
1874,2017-01-09,2.269
...,...,...
1703,2017-09-13,2.498
1702,2017-09-14,2.496
1701,2017-09-15,2.500
1700,2017-09-18,2.504


In [14]:
snp500['Price'].values

array([], dtype=float64)

In [15]:
snp500.iloc[:,1].values

array([], dtype=float64)

In [26]:
df_coeff = pd.DataFrame(columns = ['Ticker', 'alpha', 'beta', 'r_squared', 'specific_risk'])

X = snp500.iloc[:,1].values

for ticker in tickers:
    data = data180[data180['Name'] == ticker]
    data = data.merge(snp500, left_index=True, right_on='Date')
    #data['LogReturn_snp'] = np.log(data['Price'] / data['Price'].shift(1))
    #data = data.dropna()
    y = data['LogReturn']
    X = sm.add_constant(X)
    model = sm.OLS(X, y)
    result = model.fit()
    df_coeff = df_coeff.append({'Ticker': ticker, 'alpha': result.params[0], 'beta': result.params[1], 'r_squared': result.rsquared, 'specific_risk': result.mse_resid.std(), 'systematic_risk': result.params[1] ** 2 * X.std() ** 2}, ignore_index=True)


AttributeError: 'DataFrame' object has no attribute 'append'